In [1]:
import os
import importlib
import sys
import matplotlib.pyplot as plt

# sys.path.append('/Users/aeshef/Documents/GitHub/kursach/pys/improved_pipeline')

# import grid_search
# importlib.reload(grid_search)
from pys.improved_pipeline.grid_search import run_grid_search_pipeline


# import short_selling_support
# importlib.reload(short_selling_support)
from  pys.improved_pipeline.short_selling_support import run_short_selling_pipeline

# import honest_backtest
# importlib.reload(honest_backtest)
from pys.improved_pipeline.honest_backtest import HonestBacktester

In [2]:
def run_improved_pipeline(
    data_file="/Users/aeshef/Documents/GitHub/kursach/data/df.csv",
    output_dir="/Users/aeshef/Documents/GitHub/kursach/data/improved_pipeline",
    risk_free_rate=0.075,
    run_grid_search=True,
    run_honest_backtest=True,
    run_short_selling=True
):
    """
    Запускает улучшенный пайплайн с grid search, честным бэктестом и поддержкой шортов
    """
    os.makedirs(output_dir, exist_ok=True)
    
    results = {}
    
    # 1. Grid Search для нахождения оптимальных параметров
    if run_grid_search:
        grid_search_dir = os.path.join(output_dir, 'grid_search')
        os.makedirs(grid_search_dir, exist_ok=True)
        
        print("Запуск Grid Search для поиска оптимальных параметров...")
        grid_results, best_params = run_grid_search_pipeline(
            data_file=data_file,
            output_dir=grid_search_dir,
            training_period=('2024-01-01', '2024-12-31')
        )
        
        results['grid_search'] = {
            'best_params': best_params,
            'results_file': os.path.join(grid_search_dir, 'grid_search_results.csv')
        }
    
    # 2. Честный бэктест на будущий период
    if run_honest_backtest:
        honest_backtest_dir = os.path.join(output_dir, 'honest_backtest')
        os.makedirs(honest_backtest_dir, exist_ok=True)
        
        print("Запуск честного бэктеста на будущем периоде...")
        
        # Используем лучшие параметры из Grid Search, если доступны
        best_params_file = os.path.join(output_dir, 'grid_search/best_params.json') \
            if run_grid_search and os.path.exists(os.path.join(output_dir, 'grid_search/best_params.json')) \
            else None
        
        # ИЗМЕНЕНО: используем класс HonestBacktester вместо функции run_honest_backtest
        backtester = HonestBacktester(
            data_file=data_file,
            best_params_file=best_params_file,
            train_period=('2024-01-01', '2024-12-31'),
            test_period=('2025-01-01', '2025-06-30'),
            output_dir=honest_backtest_dir,
            risk_free_rate=risk_free_rate
        )
        backtest_report = backtester.run()
        
        results['honest_backtest'] = {
            'report': backtest_report,
            'report_file': os.path.join(honest_backtest_dir, 'honest_backtest_report.md')
        }
    
    # 3. Стратегия с короткими позициями
    if run_short_selling:
        short_selling_dir = os.path.join(output_dir, 'short_selling')
        os.makedirs(short_selling_dir, exist_ok=True)
        
        print("Запуск стратегии с короткими позициями...")
        short_selling_results = run_short_selling_pipeline(
            data_file=data_file,
            output_dir=short_selling_dir,
            risk_free_rate=risk_free_rate,
            period=('2024-01-01', '2025-04-15')
        )
        
        results['short_selling'] = {
            'results': short_selling_results,
            'results_dir': short_selling_dir
        }
    
    # Сохраняем итоговый отчет
    with open(os.path.join(output_dir, 'pipeline_report.md'), 'w') as f:
        f.write("# Отчет о выполнении улучшенного пайплайна\n\n")
        
        if run_grid_search and 'grid_search' in results:
            f.write("## Grid Search\n")
            f.write(f"* Лучшие параметры сохранены в: {results['grid_search']['results_file']}\n")
            
            # Если есть доступ к лучшим параметрам, выводим их
            if 'best_params' in results['grid_search']:
                best = results['grid_search']['best_params']
                f.write("* Лучшие параметры:\n")
                for k, v in best.items():
                    if k.startswith('signal_') or k.startswith('portfolio_'):
                        f.write(f"  - {k}: {v}\n")
            f.write("\n")
        
        if run_honest_backtest and 'honest_backtest' in results:
            f.write("## Честный бэктест\n")
            f.write(f"* Полный отчет: {results['honest_backtest']['report_file']}\n")
            
            # Если есть доступ к отчету, выводим основные метрики
            if 'report' in results['honest_backtest']:
                report = results['honest_backtest']['report']
                if 'test_metrics' in report:
                    metrics = report['test_metrics']
                    f.write("* Ключевые метрики тестового периода:\n")
                    f.write(f"  - Годовая доходность: {metrics['annual_return']*100:.2f}%\n")
                    f.write(f"  - Коэффициент Шарпа: {metrics['sharpe_ratio']:.2f}\n")
                    f.write(f"  - Максимальная просадка: {metrics['max_drawdown']*100:.2f}%\n")
            f.write("\n")
        
        if run_short_selling and 'short_selling' in results:
            f.write("## Стратегия с короткими позициями\n")
            f.write(f"* Результаты сохранены в: {results['short_selling']['results_dir']}\n")
            
            # Безопасная проверка наличия метрик бэктеста
            if 'results' in results['short_selling']:
                backtest_results = results['short_selling']['results'].get('backtest')
                if backtest_results is not None and isinstance(backtest_results, dict) and 'metrics' in backtest_results:
                    metrics = backtest_results['metrics']
                    f.write("* Ключевые метрики стратегии с шортами:\n")
                    f.write(f"  - Годовая доходность: {metrics['annual_return']*100:.2f}%\n")
                    f.write(f"  - Коэффициент Шарпа: {metrics['sharpe_ratio']:.2f}\n")
                    f.write(f"  - Максимальная просадка: {metrics['max_drawdown']*100:.2f}%\n")
                else:
                    f.write("* Метрики бэктеста недоступны. Возможные причины:\n")
                    f.write("  - Недостаточно данных для расчета\n")
                    f.write("  - Проблемы с совместимостью сигналов и цен\n")
                    
                # Отчет о портфеле всегда должен быть доступен
                if 'portfolio' in results['short_selling']['results']:
                    portfolio = results['short_selling']['results']['portfolio']
                    if portfolio:
                        f.write("* Состав портфеля с короткими позициями:\n")
                        for ticker, weight in portfolio.get('weights', {}).items():
                            if abs(weight) >= 0.05:  # Показываем только значимые позиции
                                f.write(f"  - {ticker}: {weight*100:.1f}%\n")
    
    print(f"Улучшенный пайплайн завершен. Результаты сохранены в: {output_dir}")
    return results


In [3]:
run_improved_pipeline(run_grid_search=True)

2025-04-25 17:37:57,876 - GridSearch - INFO - GridSearch initialized with output directory: /Users/aeshef/Documents/GitHub/kursach/data/improved_pipeline/grid_search
2025-04-25 17:37:57,879 - GridSearch - INFO - Запуск Grid Search: 432 комбинаций параметров
2025-04-25 17:37:57,879 - GridSearch - INFO - Обработка чанка 1/22 (20 комбинаций)
2025-04-25 17:37:57,880 - GridSearch - ERROR - Ошибка при тестировании комбинации параметров (ID: 0_2163108051425861148_7092693022303937636_0.075): expected str, bytes or os.PathLike object, not NoneType
2025-04-25 17:37:57 - SignalGenerator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-25 17:37:57,880 - GridSearch - ERROR - Ошибка при тестировании комбинации параметров (ID: 0_2163108051425861148_-2606967420430360810_0.075): expected str, bytes or os.PathLike object, not NoneType
2025-04-25 17:37:57 - SignalGenerator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-25 17:37:57,880 - GridSearch - ERROR - Ошибка при тестиров

Запуск Grid Search для поиска оптимальных параметров...


2025-04-25 17:37:58,063 - GridSearch - ERROR - Ошибка при тестировании комбинации параметров (ID: 0_3153119172003118295_6052817518264534396_0.075): expected str, bytes or os.PathLike object, not NoneType
2025-04-25 17:37:58 - SignalGenerator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-25 17:37:58,064 - GridSearch - ERROR - Ошибка при тестировании комбинации параметров (ID: 0_-1850802901883374829_-9023268608642359032_0.075): expected str, bytes or os.PathLike object, not NoneType
2025-04-25 17:37:58 - SignalGenerator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-25 17:37:58,130 - GridSearch - ERROR - Ошибка при тестировании комбинации параметров (ID: 0_-1850802901883374829_7092693022303937636_0.075): expected str, bytes or os.PathLike object, not NoneType
2025-04-25 17:37:58 - SignalGenerator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-25 17:37:58,164 - GridSearch - ERROR - Ошибка при тестировании комбинации параметров (ID: 0_-18508029

KeyboardInterrupt: 